In [4]:
import numpy as np
import pandas as pd
from scipy.optimize import Bounds
from copy import copy
from scipy.optimize import OptimizeResult, minimize
from optimparallel import minimize_parallel
from montecarlo import MonteCarlo
from CIRModel import BasicCIRModel

In [5]:
class Calibration:
    """Class for calibrating the classic CIR model using numerical optimization."""
    
    def __init__(self, data, dt, model_class, initial_params):
        self.data = data
        self.dt = dt
        self.model_class = model_class
        self.initial_params = initial_params

    def _calculate_error(self, params, Ts, prices):
        model = self.model_class(*params)  # Initialize the model with updated parameters
        errors = []
        for T, market_price in zip(Ts, prices):
            simulated_price = model.exact_solution(0.03, T)  # Assuming 0.03 as the initial rate
            errors.append((simulated_price - market_price)**2)
        return np.sqrt(np.mean(errors))

    def calibrate(self, bounds, params):
        initial_values = [self.initial_params[param] for param in params]
        result = minimize(
            lambda params: self._calculate_error(params, self.data['Maturity'].values, self.data['Price'].values),
            initial_values,
            method='L-BFGS-B',
            bounds=bounds)
        return result

In [7]:
if __name__ == "__main__":
    # Load and prepare yield curve data
    df = pd.read_csv("DGS_30.csv", index_col=0)
    df.index = pd.to_datetime(df.index)
    df.index.name = 'DATE'
    selected_date = '2024-03-15'
    yields = df.loc[selected_date].astype(float) / 100

    yield_data = pd.DataFrame({
        'Yield': yields.values,
        'Maturity': np.arange(1, len(yields) + 1)
    })

    yield_data["Price"] = np.exp(-yield_data["Yield"] * yield_data["Maturity"])

    # Initialize model parameters
    initial_model_params = {
        "kappa": 0.5,
        "mu_r": 0.03,
        "sigma": 0.03
    }

    # Define bounds for the parameters
    bounds = Bounds([0.001, 0.001, 0.001], [5, 1, 0.5])

    # Calibrate model
    calibrator = Calibration(
        data=yield_data,
        dt=1/252,
        model_class=BasicCIRModel,
        initial_params=initial_model_params
    )

    # Parameters to calibrate
    params = ["kappa", "mu_r", "sigma"]
    result = calibrator.calibrate(bounds=bounds, params=params)

    print("Optimized Parameters:")
    for name, value in zip(params, result.x):
        print(f"{name}: {value:.4f}")
    print(f"\nObjective Function Value: {result.fun:.6f}")
    print(f"Success: {result.success}")
    print(f"Message: {result.message}")

Optimized Parameters:
kappa: 1.2310
mu_r: 0.0459
sigma: 0.1410

Objective Function Value: 0.007158
Success: True
Message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
